In [1]:
import networkx as nx


def read_dimacs_graph(f):
    g = nx.Graph()
    expected_num_nodes = None
    expected_num_edges = None

    with open(f) as of:
        for line in of:
            if len(line) > 1:
                if line.startswith('c'):
                    pass
                elif line.startswith("p"):
                    _, file_type, num_nodes, num_edges = line.strip().split(" ")
                    expected_num_nodes = int(num_nodes)
                    expected_num_edges = int(num_edges)
                elif line.startswith("e"):
                    _, u,v = line.strip().split(" ")
                    g.add_edge(u, v)
                else:
                    raise ValueError(f"Unknown line start {line[0]}")

    #assert expected_num_nodes == g.number_of_nodes(), f"{expected_num_nodes} {g.number_of_nodes()}"
    #assert expected_num_edges == g.number_of_edges(), f"{expected_num_edges} {g.number_of_edges()}"
    return g

<font size="5">

**GeCO**

**Ge**nerators for **C**ombinatorial **O**ptimization

Common tasks when reproducing CO Papers:
- Download and read instances
- Create instances of the same type
    
Usually there is no single best file format
    
GeCO offers file format agnostic reading and writing by using popular open source libraries


<font size="5">

**Files to MIPS**

Graph file -> MIP

Making use of NetworkX and PySCIPOpt

In [2]:
# Reading a graph with networkx
graph_1 = read_dimacs_graph("../data/local/1-FullIns_4.col")

In [3]:
# Creating a (SCIP-)MIP model
import geco.mips.graph_coloring as gc

model = gc.partial_ordering(graph_1, color_upperbound=8)

In [4]:
# Solving it
model.optimize()
print(model.getStatus(), model.getObjVal(), model.getNNodes())

optimal 5.0 118


In [5]:
# Saving it
model.writeProblem("/tmp/1-FullIns_4.pop.mps")
model.writeProblem("/tmp/1-FullIns_4.pop.lp")

wrote problem to file b'/tmp/1-FullIns_4.pop.mps'
wrote problem to file b'/tmp/1-FullIns_4.pop.lp'


In [6]:
# Making use of networkx generators
graph_2 = nx.generators.erdos_renyi_graph(35, 0.2)

In [7]:
print(graph_2.number_of_nodes(), graph_2.number_of_edges())

model = gc.partial_ordering(graph_2, color_upperbound=8)
model.optimize()
print(model.getStatus(), model.getObjVal(), model.getNNodes())

35 101
optimal 4.0 1


<font size="5">

**MIPLIB functionality**

In [8]:
import geco.mips.miplib as ml

loader = ml.Loader(persistent_directory="/tmp/")

miplib_model = loader.load_instance("swath2.mps", with_solution=True)
miplib_model.optimize()

print(miplib_model.getStatus(), miplib_model.getObjVal(), miplib_model.getNNodes())

optimal 385.19969295 5280


In [15]:
import geco.mips.utilities as util

miplib_model2 = loader.load_instance("swath2.mps", with_solution=True)
miplib_model2 = util.shuffle(miplib_model2, seed=53115)

miplib_model2.optimize()

print(miplib_model2.getStatus(), miplib_model2.getObjVal(), miplib_model2.getNNodes())

wrote problem to file b'/tmp/tmpzssysx68.mps'
optimal 385.19969295 4069


<font size="5">

**Creating instances of same structure**

In [14]:
import geco.mips.knapsack as knapsack

seeds = [53115, 4711]
num_elements = 2000

for seed in seeds:
    model = knapsack.yang_instance(n=num_elements, seed=seed)
    model.writeProblem(f"/tmp/knapsack-yang_n{num_elements}seed{seed}.mps")

wrote problem to file b'/tmp/knapsack-yang_n2000seed53115.mps'
wrote problem to file b'/tmp/knapsack-yang_n2000seed4711.mps'


In [ ]:
import geco.mips.set_cover as sc

sc.yang_instance